# KNN, рак и спам
__Суммарное количество баллов: 12__

В этом домашнем задании Вам предлагается при помощи классификации методом k ближайших соседей научиться отличать тип опухоли в организме, а так же определять сообщения со спамом

In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib
import copy
import pandas
from typing import NoReturn, Tuple, List

### Задание 1 (1 балл)
Для начала работы нам необходимо научиться читать набор данных. Всего мы будем работать с двумя наборами данных.

__Cancer.csv__ - выборка данных о пациентах с доброкачественными и злокачественными опухолями. Задача - научиться их отличать.

__Spam.csv__ - набор данных большего размера. Он содержит некоторую статистику по сообщениям, а так же метку, является ли сообщение спамом. Задача - научиться автоматически отличать спам от обычных сообщений.

Реализуйте методы `read_cancer_dataset` и `read_spam_dataset`. Каждый из них принимает на вход путь к набору данных и возвращает выборку `X` и соответствующие метки `y`. Набор данных должен быть упорядочен случайно, т.е. необходимо сделать shuffle.

In [ ]:
def read_cancer_dataset(path_to_csv: str) -> Tuple[np.array, np.array]:

    """

    Parameters
    ----------
    path_to_csv : str
        Путь к cancer датасету.

    Returns
    -------
    X : np.array
        Матрица признаков опухолей.
    y : np.array
        Вектор бинарных меток, 1 соответствует доброкачественной опухоли (M),
        0 --- злокачественной (B).


    """

    df = pd.read_csv(path_to_csv)
    df = df.sample(frac=1).reset_index(drop=True)
    y = df['label']
    X = df.drop('label', axis=1)
    y = y.replace('M', 1)
    y = y.replace('B', 0)

    return np.array(X), np.array(y)

def read_spam_dataset(path_to_csv: str) -> Tuple[np.array, np.array]:

    """

    Parameters
    ----------
    path_to_csv : str
        Путь к spam датасету.

    Returns
    -------
    X : np.array
        Матрица признаков сообщений.
    y : np.array
        Вектор бинарных меток,
        1 если сообщение содержит спам, 0 если не содержит.

    """
    df = pd.read_csv(path_to_csv)
    df = df.sample(frac=1).reset_index(drop=True)
    y = df['label']
    X = df.drop('label', axis=1)

    return np.array(X), np.array(y)


In [ ]:
read_cancer_dataset("cancer.csv")
read_spam_dataset("spam.csv")

### Задание 2  (1 балл)
Начиная работать с данными, нам необходимо их предобработать и подготовить. В частности, нам необходимо разделить выборку на две: тренировочную и тестовую. Тренировочная выборка необходима для обучения алгоритма, а тестовая для проверки результатов обучения. Обычно используют коэффициент разделения `0.9`.

Необходимо вернуть кортеж из `X_train`, `y_train`, `X_test` и `y_test`

In [ ]:
def train_test_split(X: np.array, y: np.array, ratio: float) -> Tuple[np.array, np.array, np.array, np.array]:
    """

    Parameters
    ----------
    X : np.array
        Матрица признаков.
    y : np.array
        Вектор меток.
    ratio : float
        Коэффициент разделения.

    Returns
    -------
    X_train : np.array
        Матрица признаков для train выборки.
    y_train : np.array
        Вектор меток для train выборки.
    X_test : np.array
        Матрица признаков для test выборки.
    y_test : np.array
        Вектор меток для test выборки.

    """
    size = X.shape[0]
    delimiter = round(size * ratio)
    return X[:delimiter], y[:delimiter], X[delimiter:], y[delimiter:]

### Задание 3 (2 балла)
Также прежде чем приступать к решению задачи, нам необходимо определиться с метриками, которые позволят нам оценить полученное решение. Для задач классификации мы можем использовать precision, recall и accuracy. Эти метрики считаются для каждого класса.

Метод возвращает:

* Вектор __Precision__, каждый из элементов которого равен значению метрики precision для соответствующего класса.

* Вектор __Recall__, каждый из элементов которого равен значению метрики recall для соответствующего класса.

* __Accuracy__ - число, которое равно отношению правильно классифицированных элементов выборке к размеру выборки.

In [ ]:
pression, recall = [], []
classes = np.unique(y_true)
for i in range(len(classes)):
  current_class = classes[i]
  y_true_new = y_pred_new = [0] * len(y_true)
  TP = FP = TN = FN = 0
  for j in range(len(y_true)):
    if y_true[j] == current_class:
      y_true_new = 1
    if y_pred[j] == current_class:
      y_pred_new = 1
  for k in range(len(y_true)):
    if y_true_new == y_pred_new == 1:
      TP += 1
    elif y_true_new == y_pred_new == 0:
      TN += 1
    elif y_true_new != y_pred_new and y_true_new == 1:
      FP += 1
    elif y_true_new != y_pred_new and y_true_new == 0:
      FN += 1
  pression.append(TP/(TP + FP))
  recall.append(TP/(TP + TN))
accuracy = sum(y_true == y_pred) / y_pred
return np.array(pression), np.array(recall), accuracy

NameError: ignored

Теперь, имея этот метод, мы можем построить кривые зависимости Precision, Recall и Accuracy от параметра `k`

In [ ]:
def plot_precision_recall(X_train, y_train, X_test, y_test, max_k=30):
    ks = list(range(1, max_k + 1))
    classes = len(np.unique(list(y_train) + list(y_test)))
    precisions = [[] for _ in range(classes)]
    recalls = [[] for _ in range(classes)]
    accuracies = []
    for k in ks:
        classifier = KNearest(k)
        classifier.fit(X_train, y_train)
        y_pred = classifier.predict(X_test)
        precision, recall, acc = get_precision_recall_accuracy(y_pred, y_test)
        for c in range(classes):
            precisions[c].append(precision[c])
            recalls[c].append(recall[c])
        accuracies.append(acc)
    def plot(x, ys, ylabel, legend=True):
        plt.figure(figsize = (12, 3))
        plt.xlabel("K")
        plt.ylabel(ylabel)
        plt.xlim(x[0], x[-1])
        plt.ylim(np.min(ys)-0.01, np.max(ys)+0.01)
        for cls, cls_y in enumerate(ys):
            plt.plot(x, cls_y, label="Class " + str(cls))
        if legend:
            plt.legend()
        plt.tight_layout()
        plt.show()

    plot(ks, recalls, "Recall")
    plot(ks, precisions, "Precision")
    plot(ks, [accuracies], "Accuracy", legend=False)

Также для оценки качества классификации построим __ROC-кривую__. Она отражает зависимость __True Positive Rate__ (TPR) от __False Positive Rate__ (FPR) для заранее фиксированного класса. Чем график выше побочной диагонали - тем лучше.

In [ ]:
def plot_roc_curve(X_train, y_train, X_test, y_test, max_k=30):
    positive_samples = sum(1 for y in y_test if y == 0)
    ks = list(range(1, max_k + 1))
    curves_tpr = []
    curves_fpr = []
    colors = []
    for k in ks:
        colors.append([k / ks[-1], 0, 1 - k / ks[-1]])
        knearest = KNearest(k)
        knearest.fit(X_train, y_train)
        p_pred = [p[0] for p in knearest.predict_proba(X_test)]
        tpr = []
        fpr = []
        for w in np.arange(-0.01, 1.02, 0.01):
            y_pred = [(0 if p > w else 1) for p in p_pred]
            tpr.append(sum(1 for yp, yt in zip(y_pred, y_test) if yp == 0 and yt == 0) / positive_samples)
            fpr.append(sum(1 for yp, yt in zip(y_pred, y_test) if yp == 0 and yt != 0) / (len(y_test) - positive_samples))
        curves_tpr.append(tpr)
        curves_fpr.append(fpr)
    plt.figure(figsize = (7, 7))
    for tpr, fpr, c in zip(curves_tpr, curves_fpr, colors):
        plt.plot(fpr, tpr, color=c)
    plt.plot([0, 1], [0, 1], linestyle="--")
    plt.xlabel("False positive rate")
    plt.ylabel("True positive rate")
    plt.xlim(-0.01, 1.01)
    plt.ylim(-0.01, 1.01)
    plt.tight_layout()
    plt.show()

### Задание 4 (5 баллов)
Теперь приступим к реализации классификатора. В этот раз будем использовать классификацию методом k средних. Поскольку основной решаемой задачий во время классификации этим методом является поиск ближайших соседей, а набор данных может быть достаточно большим, наивная реализация будет работать очень долго.

Одним из способов решить эту проблему является __KD-дерево__. Оно позволяет значительно ускорить поиск ближайших соседей. Реализуйте построение KD-дерева и выполнение запросов на поиск k ближайших соседей.

Метод `__init__` должен принимать на вход набор точек `X`, по которому будет строиться дерево, а так же размер листов `leaf_size` построенного дерева.

Метод `query` должен принимать на вход набор точек `X`, для каждой из которых необходимо найти `k` ближайших соседей.

In [ ]:
class KDTree:
    def __init__(self, X: np.array, leaf_size: int = 40):
        """

        Parameters
        ----------
        X : np.array
            Набор точек, по которому строится дерево.
        leaf_size : int
            Минимальный размер листа
            (то есть, пока возможно, пространство разбивается на области,
            в которых не меньше leaf_size точек).

        Returns
        -------

        """
        self.leaf_size = leaf_size
        self.idx = np.arange(len(X))
        self.X_idx = np.array(list(zip(X, self.idx)), dtype=object)
        self.tree = self.build_tree(self.X_idx)

    def build_tree(self, points_idx, depth=0):
        n = points_idx.shape[0]

        if n > self.leaf_size:

            axis = depth % points_idx.shape[1]

            sorted_points_idx = sorted(points_idx, key=lambda point: point[0][axis])

            return {'point': sorted_points_idx[n // 2],
                    'left': self.build_tree(np.array(sorted_points_idx[:n // 2]), depth + 1),
                    'right': self.build_tree(np.array(sorted_points_idx[n // 2 + 1:]), depth + 1)}

        elif n >= 1:

            return {'point': points_idx,
                    'left': None,
                    'right': None}

    def merge(self, left, right):
        len_left, len_right = len(left), len(right)
        result = [0 for _ in range(len_left + len_right)]
        pA = pB = pC = 0

        while pA != len_left and pB != len_right:

            if left[pA][0] <= right[pB][0]:
                result[pC] = left[pA]
                pC += 1
                pA += 1
            else:
                result[pC] = right[pB]
                pC += 1
                pB += 1

        while pA != len_left:
            result[pC] = left[pA]
            pC += 1
            pA += 1

        while pB != len_right:
            result[pC] = right[pB]
            pC += 1
            pB += 1

        return result

    def compare_neighbours(self, distance_1, distance_2, k):
        if distance_1 is not None:
            distance_1 = sorted(distance_1, key=lambda x: x[0])
        else:
            distance_1 = []

        if distance_2 is not None:
            distance_2 = sorted(distance_2, key=lambda x: x[0])
        else:
            distance_2 = []

        return self.merge(distance_1, distance_2)[:k]

    def get_neighbours(self, tree, point_idx, k, depth=0):
        if tree is None:
            return None

        axis = depth % len(point_idx)

        if tree.get('left') is not None:

            if point_idx[0][axis] < tree.get('point')[0][axis]:
                next_tree = tree.get('left')
                opposite_tree = tree.get('right')
            else:
                next_tree = tree.get('right')
                opposite_tree = tree.get('left')

            leaf_distance = [[np.linalg.norm(point_idx[0] - tree.get('point')[0]), tree.get('point')]]

        else:
            if point_idx[0][axis] < tree.get('point')[0][0][axis]:
                next_tree = tree.get('left')
                opposite_tree = tree.get('right')
            else:
                next_tree = tree.get('right')
                opposite_tree = tree.get('left')

            leaf_distance = [[np.linalg.norm(point_idx[0] - node_el[0]), node_el] for node_el in tree.get('point')]

        distance_neighbours = self.compare_neighbours(self.get_neighbours(next_tree, point_idx, k, depth + 1),
                                                      leaf_distance, k)

        min_distance = distance_neighbours[-1][0]

        if tree.get('left') is not None:
            if min_distance > point_idx[0][axis] - tree.get('point')[0][axis]:
                distance_neighbours = self.compare_neighbours(
                    self.get_neighbours(opposite_tree, point_idx, k, depth + 1),
                    distance_neighbours, k)
        else:
            if min_distance > point_idx[0][axis] - tree.get('point')[0][0][axis]:
                distance_neighbours = self.compare_neighbours(
                    self.get_neighbours(opposite_tree, point_idx, k, depth + 1),
                    distance_neighbours, k)

        return distance_neighbours

    def query(self, X: np.array, k: int = 1):
        """

        Parameters
        ----------
        X : np.array
            Набор точек, для которых нужно найти ближайших соседей.
        k : int
            Число ближайших соседей.

        Returns
        -------
        list[list]
            Список списков (длина каждого списка k):
            индексы k ближайших соседей для всех точек из X.

        """
        ans = [[] for _ in range(len(X))]
        for idx, point in enumerate(X):

            for el in self.get_neighbours(self.tree, (point, idx), k):
                ans[idx].append(el[1][1])

        return ans



Поскольку данная струкутра данных является сложной, ее стоит протестировать отдельно. Для этого проведем тестирование с небольшим набором случайных точек. Если после выполнение вывод пуст, то KD-дерево скорее всего работает правильно.

In [ ]:
def true_closest(X_train, X_test, k):
    result = []
    for x0 in X_test:
        bests = list(sorted([(i, np.linalg.norm(x - x0)) for i, x in enumerate(X_train)], key=lambda x: x[1]))
        bests = [i for i, d in bests]
        result.append(bests[:min(k, len(bests))])
    return result

X_train = np.random.randn(100, 3)
X_test = np.random.randn(10, 3)
tree = KDTree(X_train, leaf_size=2)
predicted = tree.query(X_test, k=4, return_distance=False)
true = true_closest(X_train, X_test, k=4)

if np.sum(np.abs(np.array(np.array(predicted).shape) - np.array(np.array(true).shape))) != 0:
    print("Wrong shape")
else:
    errors = sum([1 for row1, row2 in zip(predicted, true) for i1, i2 in zip(row1, row2) if i1 != i2])
    if errors > 0:
        print("Encounted", errors, "errors")

### Задание 5  (3 балла)
Осталось реализовать сам классификатор. Реализуйте его, используя KD-дерево.

Метод `__init__` принимает на вход количество соседей, по которым предсказывается класс, и размер листьев KD-дерева.

Метод `fit` должен по набору данных и меток строить классификатор.

Метод `predict_proba` должен предсказывать веротности классов для заданного набора данных основываясь на классах соседей

In [ ]:
class KNearest:
    def __init__(self, n_neighbors: int = 5, leaf_size: int = 30):
        """

        Parameters
        ----------
        n_neighbors : int
            Число соседей, по которым предсказывается класс.
        leaf_size : int
            Минимальный размер листа в KD-дереве.

        """
        self.n_neighbors = n_neighbors
        self.leaf_size = leaf_size

    def fit(self, X: np.array, y: np.array):
        self.tree = KDTree(X, self.leaf_size)
        self.y = y
        """

        Parameters
        ----------
        X : np.array
            Набор точек, по которым строится классификатор.
        y : np.array
            Метки точек, по которым строится классификатор.

        """

    def predict_proba(self, X: np.array):

        """

        Parameters
        ----------
        X : np.array
            Набор точек, для которых нужно определить класс.

        Returns
        -------
        list[np.array]
            Список np.array (длина каждого np.array равна числу классов):
            вероятности классов для каждой точки X.


        """

        neighbors = self.tree.query(X, self.n_neighbors)
        count_clazz = np.unique(self.y).shape[0]
        ans = [[0 for _ in range(count_clazz)] for _ in range(X.shape[0])]

        for idx, neighbor in enumerate(neighbors):
            for el in neighbor:
                clazz = self.y[el]
                ans[idx][clazz] += 1

        for idx, el in enumerate(ans):
            ans[idx] = [x / count_clazz for x in el]

        return ans

    def predict(self, X: np.array) -> np.array:

        """

        Parameters
        ----------
        X : np.array
            Набор точек, для которых нужно определить класс.

        Returns
        -------
        np.array
            Вектор предсказанных классов.


        """
        return np.argmax(self.predict_proba(X), axis=1)

Наконец, протестируем наш классификатор на различных наборах данных. Реализация KNearest должна отработать за разумное время.

In [ ]:
X, y = read_cancer_dataset("cancer.csv")
X_train, y_train, X_test, y_test = train_test_split(X, y, 0.9)
plot_precision_recall(X_train, y_train, X_test, y_test)
plot_roc_curve(X_train, y_train, X_test, y_test, max_k=10)

In [ ]:
X, y = read_spam_dataset("spam.csv")
X_train, y_train, X_test, y_test = train_test_split(X, y, 0.9)
plot_precision_recall(X_train, y_train, X_test, y_test, max_k=20)
plot_roc_curve(X_train, y_train, X_test, y_test, max_k=20)